In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import math
import mat73
import warnings
import numpy as np
from tqdm import trange
import matplotlib.pyplot as plt
from scipy.optimize import nnls
from sklearn.decomposition import NMF
from scipy.io import loadmat, savemat
from matplotlib.patches import Rectangle
from sklearn.decomposition import FastICA
from matplotlib.colors import ListedColormap
from utils import *

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
abs_coeff, skin_dict, water_dict = {}, {}, {}
wave_abs = np.load('../data/hbo2hbchpr_57.npy')
plt3d = loadmat('../data/3DPlot.mat')
X, Y = plt3d['x'], plt3d['y']
abs_dict = {}

for idx, wave in enumerate(np.arange(700, 981, 5)):
    abs_coeff[wave] = wave_abs[idx]
    skin = (0.244 + 85.3 * math.exp(-(wave - 154)/66.2)) / 100
    melanin = (1.70 * 1e12 * math.pow(wave, -3.48)) / 100
    skin_dict[wave] = np.array([skin, melanin])
    abs_dict[int(wave)] = list(np.append(np.append(wave_abs[idx], skin), melanin))

wave_list = [750, 800, 850]
coeffs = np.vstack([abs_coeff[wave] for wave in wave_list])
fcoeffs = np.vstack([np.append(abs_coeff[wave], skin_dict[wave], axis = 0) for wave in wave_list])
norm_coeffs = np.array([fcoeffs.T[idx] / max(coeffs.T[idx]) for idx in range(coeffs.shape[1])]).T
legend = ['HbO', 'Hb', 'Cholesterol']
melorder = ['HbO2', 'Hb', 'Cholesterol', 'Melanin']
skinorder = ['HbO2', 'Hb', 'Cholesterol', 'Skin Baseline']
hbhbo2fat = np.copy(coeffs)[:, [0, 1, 2]]
hbhbo2fatskin = np.copy(fcoeffs[:, [0, 1, 2, 4]])
hbhbo2fatmelanin = np.copy(fcoeffs[:, [0, 1, 2, 5]])

mixed_coeffs = []
for idx in [760, 808, 915]:
    if idx == 808:
        ids = 0.69 * abs_coeff[805] + 0.31 * abs_coeff[940]
    else:
        ids = abs_coeff[idx]
    mixed_coeffs.append(ids)
mixed_coeffs = np.array(mixed_coeffs)[:, :3]

# Hypercapnia

In [ ]:
fidx = 0
dir = "../../acousticx/test/"
fdir = sorted([file for file in os.listdir(dir) if file.endswith(".mat")])
pafile = loadmat(os.path.join(dir, fdir[fidx]))['beamformed']
pafile = pafile.transpose((2, 0, 1))
paw1, paw2, frames = pafile[list(range(0, pafile.shape[0], 2))], pafile[list(range(1, pafile.shape[0], 2))], pafile.shape[0]
rois = textparse(os.path.join(dir, fdir[0].split(".")[0] + ".txt"))

In [ ]:
waves = [760, 808]
nrois = round(len(rois) / 2)
ratios, imgfile = 2 * [(0, 0.2), (0.2, 0.8), (0.8, 1)], []
wave_dict = {760: 0, 808: 1, 915: 2}
wave_list = [wave_dict[idx] for idx in waves]
swv = [str(wave) if wave != 808 else "808/940" for wave in waves]
title = [f'PRE CO2 {swv[0]}NM', f'CO2 {swv[0]}NM', f'POST CO2 {swv[0]}NM', f'PRE CO2 {swv[1]}NM', f'CO2 {swv[1]}NM', f'POST CO2 {swv[1]}NM']
for idx in range(len(ratios)):
    if idx < 3:
        imgfile.append(np.mean(paw1[round(ratios[idx][0] * frames / 2): round(ratios[idx][1] * frames / 2), rois[-1][0]: rois[-1][1], rois[-1][2]: rois[-1][3]], axis = 0))
    else:
        imgfile.append(np.mean(paw2[round(ratios[idx][0] * frames / 2): round(ratios[idx][1] * frames / 2), rois[-1][0]: rois[-1][1], rois[-1][2]: rois[-1][3]], axis = 0))
imgfile = np.array(imgfile)
plt.figure(figsize = (12, 8))
for idx in range(len(imgfile)):
    plt.subplot(2, 3, idx + 1)
    if idx < 3:
        plt.imshow(imgfile[idx], cmap = 'hot', extent = [0, 1, 0, 1])
        # plt.clim([0, 4e6])
    else:
        plt.imshow(imgfile[idx], cmap = 'hot', extent = [0, 1, 0, 1])
        # plt.clim([0, 1e6])
    plt.colorbar()
    plt.title(title[idx])

In [ ]:
title_order = ['PRE CO2', 'CO2', 'POST CO2']
order = ['HbO', 'Hb', 'Cholesterol']
imgdict, didx = {}, 0
for i in [[0, 1], [0, 2], [1, 2]]:
    for idx, j in enumerate([[0, 3], [1, 4], [2, 5]]):
        imgdict[didx] = (title_order[idx], i, j)
        didx += 1
imgidx = 1
plt.figure(figsize = (24, 10))
for idx in range(9):
    exp_unmixed = run_linear_unmixing(normalize(imgfile[imgdict[idx][2], :, :].copy()), mixed_coeffs[wave_list][:, imgdict[idx][1]])
    for xidx in range(2):
        plt.subplot(3, 6, imgidx)
        plt.imshow(exp_unmixed[:, :, xidx], cmap = "hot", extent = [0, 1, 0, 1])
        plt.colorbar()
        plt.title(f"{imgdict[idx][0]} {order[imgdict[idx][1][xidx]]}")
        # plt.clim([0, np.max(exp_unmixed)])
        imgidx += 1
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize = (15, 4))
for idx in range(3):
    exp_unmixed = run_linear_unmixing(normalize(imgfile[imgdict[idx][2], :, :].copy()), mixed_coeffs[wave_list][:, imgdict[idx][1]])
    plt.subplot(1, 3, idx + 1)
    plt.imshow(exp_unmixed[:, :, 0] / (exp_unmixed[:, :, 0] + exp_unmixed[:, :, 1]), cmap = 'jet', extent = [0, 1, 0, 1])
    plt.colorbar()
    plt.title(f"{imgdict[idx][0]}")
    plt.clim([0.6, 1])
plt.show()

In [ ]:
plt.figure(figsize = (25, 8))
plt.subplot(2, 5, 1)
plt.imshow(paw1[0, rois[-1][0]: rois[-1][1], rois[-1][2]: rois[-1][3]], cmap = "hot", extent = [0, 1, 0, 1])
plt.title(f"{swv[0]} ROIs")
plt.colorbar()
plt.subplot(2, 5, 6)
plt.imshow(paw2[0, rois[-1][0]: rois[-1][1], rois[-1][2]: rois[-1][3]], cmap = "hot", extent = [0, 1, 0, 1])
plt.title(f"{swv[1]} ROIs")
plt.colorbar()
for idx, subidx in enumerate(list(set(list(range(10))) - set([0, 5]))):
    plt.subplot(2, 5, subidx + 1)
    if idx < nrois:
        mean = np.mean(paw1[:, rois[idx][0]:rois[idx][1], rois[idx][2]:rois[idx][3]], axis = (1, 2))
    else:
        mean = np.mean(paw2[:, rois[idx][0]:rois[idx][1], rois[idx][2]:rois[idx][3]], axis = (1, 2))
    mavg = moving_average(mean, 5)
    plt.plot(mean, "b")
    plt.plot(mavg, 'r')
    plt.axvspan(round(frames * ratios[1][0] / 2), round(frames * ratios[1][1] / 2), color = 'coral', alpha = 0.4, lw = 0)
    plt.title(f'{waves[0] if idx < len(rois) else waves[1]} NM ROI - {(idx + 1) if idx < 4 else (idx - 3)}')
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 5, figsize = (25, 8))
roiplot(axs[0, 0], paw1[0, rois[-1][0]:rois[-1][1], rois[-1][2]:rois[-1][3]], rois[: 4], title = f"{swv[0]} ROIs")
roiplot(axs[1, 0], paw2[0, rois[-1][0]:rois[-1][1], rois[-1][2]:rois[-1][3]], rois[4:8], title = f"{swv[1]} ROIs")
for idx, subidx in enumerate(list(set(list(range(10))) - set([0, 5]))):
    ax = axs[subidx // 5, subidx % 5]
    if idx < nrois:
        mean = np.mean(paw1[:, rois[idx][0]:rois[idx][1], rois[idx][2]:rois[idx][3]], axis=(1, 2))
    else:
        mean = np.mean(paw2[:, rois[idx][0]:rois[idx][1], rois[idx][2]:rois[idx][3]], axis=(1, 2))
    mavg = moving_average(mean, 5)
    ax.plot(mean, "b")
    ax.plot(mavg, 'r')
    ax.axvspan(round(frames * ratios[1][0] / 2), round(frames * ratios[1][1] / 2), color='coral', alpha=0.4, lw=0)
    ax.set_title(f'{swv[0] if idx < nrois else swv[1]} NM ROI - {(idx + 1) if idx < 4 else (idx - 3)}')
plt.show()